# LLM and Prompting Basics

## Basic LLM Call and Components

1) LLM Access Authorization setup
2) Prompt Template
3) Messages:
    - System Prompt
    - User Prompt
    - Assistant Prompt (AI prompt)
4) LLM Function call


## UseCase: Social Media Post Generator
### 1) LLM Access Authorization

In [ ]:
# api_key = "YOUR API KEY"

In [2]:
# LLM API Access setup
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = api_key

### 2) Prompt Template
- Template to collate static and dynamic prompts


In [6]:
# input data
product_name = "EcoLight Smart Bulb"
cool_feature = "Energy-saving with customizable colors and voice control"
audience_persona = "Environmentally conscious homeowners"

# Social Media Marketing Assistant
prompt_template = f"""Create a catchy, clever 140 character social media post targeted toward {audience_persona} introducing and promoting a new {cool_feature} feature of {product_name}."""

### 3) Messages
    - System Prompt : Set global AI behavior
    - User Prompt : Prompts from the user
    - Assistant Prompt (AI prompt) : Responses from the LLM

In [5]:
system_prompt = "You are a social media influencer and writer."
user_prompt = prompt_template.format(audience_persona, cool_feature, product_name)
user_prompt

'Create a catchy, clever 140 character social media post targeted toward Environmentally conscious homeowners introducing and promoting a new Energy-saving with customizable colors and voice control feature of EcoLight Smart Bulb.'

In [7]:
messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": user_prompt
          }
          ]

### 4) LLM Function call

**Function Call Arguments**
- temperature:	Controls randomness: higher = more creative, lower = more focused.
- top_p: 	Nucleus sampling: limits token choices to top probability mass p.
- max_tokens:	Limits length of the generated response (in tokens).
- frequency_penalty:	Penalizes repeated tokens to reduce verbosity/repetition.
- presence_penalty:	Encourages introducing new topics by discouraging reuse of existing ones.
- messages:	Provides chat history as a list of user/assistant/system turns.
- model:	Chooses which LLM to use (e.g., gpt-4, gpt-3.5-turbo).

In [8]:
# Function to call the OpenAI GPT-3.5 API
def get_response(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=1,
            max_tokens=256,
            top_p=1
        )
        # The response is a JSON object containing more information than the generated post. We want to return only the message content
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

# Generating the social media post
generated_post = get_response(prompt_template)

# Printing the output. 
print("Generated Social Media Post:")
print(generated_post)

Generated Social Media Post:
"🌿💡 Illuminate your home eco-consciously with the new EcoLight Smart Bulb! Save energy, set customizable colors, all with voice control. Your light, your way! #EcoFriendly #SmartHome"


---
---

## Prompt Engineering Best Practices

1) Be Specific and Clear
2) Use Delimiters to set focus
3) Explicitly state output requirements and format
4) For Complex tasks, request step by step
5) Role Playing method

We first load the necessary libraries:

In [10]:
# Basic completion function
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

### 1) Be Specific and Clear

Write instructions as clear and specific as possible to get the desired LLM behaviors:

In [15]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommend movies to a customer.
You are responsible to recommend a movie from a list of top global trending movies like {global_trending_movies}.
If the customer specifies a type of movie then pick the movie from the list that closely matches customer request. 
You should refrain from asking users for their preferences and avoid asking for personal information.
If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """Please recommend a movie based on my interests."""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

Sorry, couldn't find a movie to recommend today.


- Specific request as below are better than generic response like above

In [16]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommends movies to a customer.
You are responsible to recommend a movie from a list of top global trending movies from {global_trending_movies}.
If the customer specifies a type of movie then pick the movie from the list that closely matches customer request. 
You should refrain from asking users for their preferences and avoid asking for personal information.
If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """I love super-hero movies. Please recommend a movie based on my interests."""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

I recommend you watch "The Suicide Squad" as it is a popular superhero movie from the list of top global trending movies. Enjoy!


### 2) Add Delimiters

- delimiters help to focus LLM to a specific part of the prompt

In [17]:
import IPython

prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

IPython.display.Markdown("```python" + get_completion(message) + "\n```")

```python```python
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
strings2.append("4")
```
```

### 3) Explicitly state output requirements and format

LLM are stochastic in the output format unless it has prompts that specify the output hence explicitly state how you want your output.  

In [18]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.
Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any)
###
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

print(get_completion(message))

{
    "product_name": "Nike Air Max 270 React",
    "product_brand": "Nike"
}


### 4) For Complex tasks, request step by step

To elicit reasoning in LLMs, you can prompt the model to think step-by-step. Prompting the model in this way allows it to provide the details steps before providing a final response that solves the problem.

In [19]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "system",
        "content": prompt
    }
]

response= get_completion(messages)

print(response)

The odd numbers in the group are 15, 5, 13, 7, and 1.

Adding these odd numbers together: 15 + 5 + 13 + 7 + 1 = 41.

Since 41 is an odd number, the sum of the odd numbers in the group is odd.


### 5) Role Playing method

Role Playing is another technique to elicit a specific behavior. 

In [21]:
system_message = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
"""

user_message_1 = """
Hello, who are you?
"""

ai_message_1 = """
Greeting! I am an AI research assistant. How can I help you today?
"""

prompt = """
Human: Can you tell me about the creation of blackholes?
AI:
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": ai_message_1

    },
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(messages)
print(response)

Black holes are formed when massive stars exhaust their nuclear fuel and undergo a supernova explosion. If the core of the star is massive enough, it collapses under its own gravity, forming a singularity - a point of infinite density. This singularity is surrounded by an event horizon, beyond which nothing, not even light, can escape. This creates a region of spacetime with extremely strong gravitational effects, known as a black hole.
